# Finding Synonyms and Analogies

This notebook is taken from a [PyTorch NLP tutorial](https://github.com/joosthub/pytorch-nlp-tutorial-ny2018/blob/master/day_1/0_Using_Pretrained_Embeddings.ipynb) source: [repository for the training tutorial as the 2018 O'Reilly AI Conference in NYC on April 29 and 30, 2018](https://github.com/joosthub/pytorch-nlp-tutorial-ny2018)

Another related sources [chapter 10.6](http://www.d2l.ai/chapter_natural-language-processing/similarity-analogy.html) of [Dive into Deep Learning](http://www.d2l.ai/index.html) and [chapter 10.5](http://www.d2l.ai/chapter_natural-language-processing/glove.html) explaining Word Embedding with Global Vectors (GloVe)

In [1]:
# you might need to install annoy with the following command:
# conda install -c conda-forge python-annoy
from annoy import AnnoyIndex
import numpy as np
import torch
from tqdm import tqdm_notebook
from argparse import Namespace

Glove embeddings can be downloaded from [GloVe webpage](https://nlp.stanford.edu/projects/glove/).

Given the sizes of the files, it is better not to dowload it through WIFI!

In [2]:
# this need to be modified!
args = Namespace(
    glove_filename='/home/lelarge/data/glove/glove.6B.100d.txt'
)

In [3]:
def load_word_vectors(filename):
    word_to_index = {}
    word_vectors = []
    
    with open(filename) as fp:
        for line in tqdm_notebook(fp.readlines(), leave=False):
            line = line.split(" ")
            
            word = line[0]
            word_to_index[word] = len(word_to_index)
            
            vec = np.array([float(x) for x in line[1:]])
            word_vectors.append(vec)
            
    return word_to_index, word_vectors

In [4]:
word_to_index, word_vectors = load_word_vectors(args.glove_filename)

In [5]:
len(word_vectors)

400000

In [6]:
word_vectors[0].shape

(100,)

In [7]:
word_to_index['beautiful']

3366

In [8]:
class PreTrainedEmbeddings(object):
    def __init__(self, glove_filename):
        self.word_to_index, self.word_vectors = load_word_vectors(glove_filename)
        self.word_vector_size = len(self.word_vectors[0])
        
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}
        self.index = AnnoyIndex(self.word_vector_size, metric='euclidean')
        print('Building Index')
        for _, i in tqdm_notebook(self.word_to_index.items(), leave=False):
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print('Finished!')
    
    def get_embedding(self, word):
        return self.word_vectors[self.word_to_index[word]]
    
    def closest(self, word, n=1):
        vector = self.get_embedding(word)
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def closest_v(self, vector, n=1):
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def sim(self, w1, w2):
        return np.dot(self.get_embedding(w1), self.get_embedding(w2))


In [9]:
glove = PreTrainedEmbeddings(args.glove_filename)

Building Index


Finished!


In [10]:
glove.closest('apple', n=5)

['apple', 'microsoft', 'pc', 'software', 'google']

In [11]:
glove.closest('chip', n=5)

['chip', 'chips', 'semiconductor', 'intel', 'makers']

In [12]:
glove.closest('baby', n=5)

['baby', 'boy', 'girl', 'mom', 'child']

In [13]:
glove.closest('beautiful', n=5)

['beautiful', 'lovely', 'wonderful', 'brilliant', 'handsome']

In [14]:
def SAT_analogy(w1, w2, w3):
    '''
    Solves problems of the type:
    w1 : w2 :: w3 : __
    '''
    closest_words = []
    try:
        w1v = glove.get_embedding(w1)
        w2v = glove.get_embedding(w2)
        w3v = glove.get_embedding(w3)
        w4v = w3v + (w2v - w1v)
        closest_words = glove.closest_v(w4v, n=5)
        closest_words = [w for w in closest_words if w not in [w1, w2, w3]]
    except:
        pass
    if len(closest_words) == 0:
        print(':-(')
    else:
        print('{} : {} :: {} : {}'.format(w1, w2, w3, closest_words[0]))

In [15]:
SAT_analogy('man', 'he', 'woman')

man : he :: woman : she


In [16]:
SAT_analogy('fly', 'plane', 'sail')

fly : plane :: sail : ship


In [17]:
SAT_analogy('beijing', 'china', 'tokyo')

beijing : china :: tokyo : japan


In [18]:
SAT_analogy('man', 'woman', 'son')

man : woman :: son : daughter


In [19]:
SAT_analogy('man', 'leader', 'woman')

man : leader :: woman : opposition


In [20]:
SAT_analogy('woman', 'leader', 'man')

woman : leader :: man : leaders
